In [2]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.feature_selection import RFE,SequentialFeatureSelector
from tqdm import tqdm
from imblearn.pipeline import Pipeline

import sys
import warnings
if not sys.warnoptions:
    warnings.simplefilter("ignore")
    
from sklearn.naive_bayes import GaussianNB
rs = 0

In [3]:
processed_train_data = pd.read_csv("Processed_train_data.csv")
processed_test_data = pd.read_csv("Processed_test_data.csv")

In [4]:
processed_train_data.head()

,duration,protocol_type_icmp,protocol_type_tcp,protocol_type_udp,service,flag,src_bytes,dst_bytes,land,wrong_fragment,...,diff_srv_rate,srv_diff_host_rate,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_srv_serror_rate,class
0,0.0,0.0,1.0,0.0,0.273363,0.145284,0.009003,0.000000,0.0,0.0,...,0.00,0.00,0.588235,0.098039,0.17,0.03,0.17,0.00,0.00,0
1,0.0,0.0,0.0,1.0,0.389904,0.145284,0.002677,0.000000,0.0,0.0,...,0.15,0.00,1.000000,0.003922,0.00,0.60,0.88,0.00,0.00,0
2,0.0,0.0,1.0,0.0,0.953923,0.989729,0.000000,0.000000,0.0,0.0,...,0.07,0.00,1.000000,0.101961,0.10,0.05,0.00,0.00,1.00,1
3,0.0,0.0,1.0,0.0,0.056722,0.145284,0.004254,0.329867,0.0,0.0,...,0.00,0.00,0.117647,1.000000,1.00,0.00,0.03,0.04,0.01,0
4,0.0,0.0,1.0,0.0,0.056722,0.145284,0.003649,0.016993,0.0,0.0,...,0.00,0.09,1.000000,1.000000,1.00,0.00,0.00,0.00,0.00,0


In [5]:
processed_train_data_trX = processed_train_data.drop(columns=["class"],axis=1)
processed_train_data_trY = processed_train_data["class"]

In [6]:
processed_test_data_trX = processed_test_data.drop(columns=["class"],axis=1)
processed_test_data_trY = processed_test_data["class"]

In [7]:
from imblearn.over_sampling import ADASYN, SMOTE
adasyn = ADASYN(random_state=rs)
smote = SMOTE(random_state=rs)

processed_smote_train_data_trX, processed_smote_train_data_trY = smote.fit_resample(processed_train_data_trX, processed_train_data_trY)

In [8]:
processed_adasyn_train_data_trX, processed_adasyn_train_data_trY = adasyn.fit_resample(processed_train_data_trX, processed_train_data_trY)

In [9]:
processed_smote_train_data = processed_smote_train_data_trX
processed_smote_train_data["class"] = processed_smote_train_data_trY

In [10]:
processed_smote_train_data["class"].value_counts()

0    64928
1    64928
Name: class, dtype: int64

In [11]:
processed_smote_train_data.to_csv("processed_smote_train_data.csv",index=False)

In [12]:
processed_adasyn_train_data = processed_adasyn_train_data_trX
processed_adasyn_train_data["class"] = processed_adasyn_train_data_trY

In [13]:
processed_adasyn_train_data["class"].value_counts()

0    64928
1    64813
Name: class, dtype: int64

In [14]:
processed_train_data["class"].value_counts()

0    64928
1    57709
Name: class, dtype: int64

In [15]:
processed_adasyn_train_data.to_csv("processed_adasyn_train_data.csv",index=False)

In [16]:
processed_smote_train_data_trX = processed_smote_train_data.drop(columns=["class"],axis=1)
processed_smote_train_data_trY = processed_smote_train_data["class"]

processed_adasyn_train_data_trX = processed_adasyn_train_data.drop(columns=["class"],axis=1)
processed_adasyn_train_data_trY = processed_adasyn_train_data["class"]

In [17]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, plot_confusion_matrix,f1_score,precision_score,recall_score,RocCurveDisplay,PrecisionRecallDisplay

In [40]:
#Hyperparameter Tuning of Naive Bayes Classifier
from sklearn.model_selection import GridSearchCV
gnb = GaussianNB()
params = {
    'var_smoothing': [0.02310129700083159]
}
grid_search = GridSearchCV(estimator=gnb, 
                           param_grid=params, 
                           cv=5, n_jobs=-1, verbose=3, scoring = "f1")
grid_search.fit(processed_train_data_trX,processed_train_data_trY)
grid_search.best_score_

Fitting 5 folds for each of 1 candidates, totalling 5 fits


0.9579125819036907

In [41]:
#Hyperparameter Tuning of Naive Bayes Classifier
from sklearn.model_selection import GridSearchCV
gnb = GaussianNB()
params = {
    'var_smoothing': [0.02310129700083159]
}
grid_search = GridSearchCV(estimator=gnb, 
                           param_grid=params, 
                           cv=5, n_jobs=-1, verbose=3, scoring = "f1")
grid_search.fit(processed_smote_train_data_trX,processed_smote_train_data_trY)
grid_search.best_score_

Fitting 5 folds for each of 1 candidates, totalling 5 fits


0.9607588809467945

In [42]:
#Hyperparameter Tuning of Naive Bayes Classifier
from sklearn.model_selection import GridSearchCV
gnb = GaussianNB()
params = {
    'var_smoothing': [1e-09]
}
grid_search = GridSearchCV(estimator=gnb, 
                           param_grid=params, 
                           cv=5, n_jobs=-1, verbose=3, scoring = "f1")
grid_search.fit(processed_adasyn_train_data_trX,processed_adasyn_train_data_trY)
grid_search.best_score_

Fitting 5 folds for each of 1 candidates, totalling 5 fits


0.928302574273388

In [34]:
from sklearn.tree import DecisionTreeClassifier

In [44]:
#Hyperparameter Tuning of Decision Tree Classifier
from sklearn.model_selection import GridSearchCV
rs = 0

dt = DecisionTreeClassifier(random_state=rs)
params = {
    'max_depth': [10],
    'min_samples_leaf': [5],
    'criterion': ["entropy"]
}
grid_search = GridSearchCV(estimator=dt, 
                           param_grid=params, 
                           cv=5, n_jobs=-1, verbose=1, scoring = "f1")
grid_search.fit(processed_train_data_trX,processed_train_data_trY)
print(grid_search.best_score_)

Fitting 5 folds for each of 1 candidates, totalling 5 fits
0.9980324538371492


In [45]:
#Hyperparameter Tuning of Decision Tree Classifier
from sklearn.model_selection import GridSearchCV
rs = 0

dt = DecisionTreeClassifier(random_state=rs)
params = {
    'max_depth': [20],
    'min_samples_leaf': [5],
    'criterion': ["entropy"]
}
grid_search = GridSearchCV(estimator=dt, 
                           param_grid=params, 
                           cv=5, n_jobs=-1, verbose=1, scoring = "f1")
grid_search.fit(processed_smote_train_data_trX,processed_smote_train_data_trY)
print(grid_search.best_score_)

Fitting 5 folds for each of 1 candidates, totalling 5 fits
0.9980049698317799


In [46]:
#Hyperparameter Tuning of Decision Tree Classifier
from sklearn.model_selection import GridSearchCV
rs = 0

dt = DecisionTreeClassifier(random_state=rs)
params = {
    'max_depth': [20],
    'min_samples_leaf': [5],
    'criterion': ["entropy"]
}
grid_search = GridSearchCV(estimator=dt, 
                           param_grid=params, 
                           cv=5, n_jobs=-1, verbose=1, scoring = "f1")
grid_search.fit(processed_adasyn_train_data_trX,processed_adasyn_train_data_trY)
print(grid_search.best_score_)

Fitting 5 folds for each of 1 candidates, totalling 5 fits
0.9815950274308483


In [47]:
from sklearn.neighbors import KNeighborsClassifier
#Hyperparameter Tuning of K Nearest Neighbor
from sklearn.model_selection import GridSearchCV
rs = 0

knn = KNeighborsClassifier()
params = {
    'n_neighbors': [1]
}
grid_search = GridSearchCV(estimator=knn, 
                           param_grid=params, 
                           cv=5, verbose=10, scoring = "f1")
grid_search.fit(processed_train_data_trX,processed_train_data_trY)
print(grid_search.best_score_)

Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV 1/5; 1/1] START n_neighbors=1...............................................
[CV 1/5; 1/1] END ................n_neighbors=1;, score=0.997 total time=  20.4s
[CV 2/5; 1/1] START n_neighbors=1...............................................
[CV 2/5; 1/1] END ................n_neighbors=1;, score=0.997 total time=  19.3s
[CV 3/5; 1/1] START n_neighbors=1...............................................
[CV 3/5; 1/1] END ................n_neighbors=1;, score=0.997 total time=  20.4s
[CV 4/5; 1/1] START n_neighbors=1...............................................
[CV 4/5; 1/1] END ................n_neighbors=1;, score=0.998 total time=  20.7s
[CV 5/5; 1/1] START n_neighbors=1...............................................
[CV 5/5; 1/1] END ................n_neighbors=1;, score=0.997 total time=  19.7s
0.997418003327534


In [48]:
#Hyperparameter Tuning of Decision Tree Classifier
from sklearn.model_selection import GridSearchCV
rs = 0

knn = KNeighborsClassifier()
params = {
    'n_neighbors': [1]
}
grid_search = GridSearchCV(estimator=knn, 
                           param_grid=params, 
                           cv=5, verbose=10, scoring = "f1")
grid_search.fit(processed_smote_train_data_trX,processed_smote_train_data_trY)
print(grid_search.best_score_)

Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV 1/5; 1/1] START n_neighbors=1...............................................
[CV 1/5; 1/1] END ................n_neighbors=1;, score=0.998 total time=  21.6s
[CV 2/5; 1/1] START n_neighbors=1...............................................
[CV 2/5; 1/1] END ................n_neighbors=1;, score=0.997 total time=  21.7s
[CV 3/5; 1/1] START n_neighbors=1...............................................
[CV 3/5; 1/1] END ................n_neighbors=1;, score=0.997 total time=  21.7s
[CV 4/5; 1/1] START n_neighbors=1...............................................
[CV 4/5; 1/1] END ................n_neighbors=1;, score=0.998 total time=  21.6s
[CV 5/5; 1/1] START n_neighbors=1...............................................
[CV 5/5; 1/1] END ................n_neighbors=1;, score=0.998 total time=  21.7s
0.9977436071084005


In [49]:
#Hyperparameter Tuning of Decision Tree Classifier
from sklearn.model_selection import GridSearchCV
rs = 0

knn = KNeighborsClassifier()
params = {
    'n_neighbors': [1]
}
grid_search = GridSearchCV(estimator=knn, 
                           param_grid=params, 
                           cv=5, verbose=10, scoring = "f1")
grid_search.fit(processed_adasyn_train_data_trX,processed_adasyn_train_data_trY)
print(grid_search.best_score_)

Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV 1/5; 1/1] START n_neighbors=1...............................................
[CV 1/5; 1/1] END ................n_neighbors=1;, score=0.998 total time=  21.5s
[CV 2/5; 1/1] START n_neighbors=1...............................................
[CV 2/5; 1/1] END ................n_neighbors=1;, score=0.998 total time=  22.9s
[CV 3/5; 1/1] START n_neighbors=1...............................................
[CV 3/5; 1/1] END ................n_neighbors=1;, score=0.998 total time=  22.0s
[CV 4/5; 1/1] START n_neighbors=1...............................................
[CV 4/5; 1/1] END ................n_neighbors=1;, score=0.998 total time=  21.6s
[CV 5/5; 1/1] START n_neighbors=1...............................................
[CV 5/5; 1/1] END ................n_neighbors=1;, score=0.960 total time=  21.5s
0.9906148518463043


In [50]:
from sklearn.linear_model import LogisticRegression

In [53]:
#Hyperparameter Tuning of Logistic Regression
from sklearn.model_selection import GridSearchCV
log_reg = LogisticRegression(random_state=rs)
params = [    
    {'penalty' : ['l2'],
    'C' : [29.763514416313132],
    'solver' : ['liblinear'],
    'max_iter' : [1000]
   }
]

grid_search = GridSearchCV(estimator=log_reg,
                           param_grid=params,
                           cv=5, verbose=10, scoring = "f1")
grid_search.fit(processed_train_data_trX, processed_train_data_trY)
grid_search.best_score_

Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV 1/5; 1/1] START C=29.763514416313132, max_iter=1000, penalty=l2, solver=liblinear
[CV 1/5; 1/1] END C=29.763514416313132, max_iter=1000, penalty=l2, solver=liblinear;, score=0.979 total time=   0.7s
[CV 2/5; 1/1] START C=29.763514416313132, max_iter=1000, penalty=l2, solver=liblinear
[CV 2/5; 1/1] END C=29.763514416313132, max_iter=1000, penalty=l2, solver=liblinear;, score=0.980 total time=   0.5s
[CV 3/5; 1/1] START C=29.763514416313132, max_iter=1000, penalty=l2, solver=liblinear
[CV 3/5; 1/1] END C=29.763514416313132, max_iter=1000, penalty=l2, solver=liblinear;, score=0.979 total time=   0.6s
[CV 4/5; 1/1] START C=29.763514416313132, max_iter=1000, penalty=l2, solver=liblinear
[CV 4/5; 1/1] END C=29.763514416313132, max_iter=1000, penalty=l2, solver=liblinear;, score=0.979 total time=   0.7s
[CV 5/5; 1/1] START C=29.763514416313132, max_iter=1000, penalty=l2, solver=liblinear
[CV 5/5; 1/1] END C=29.763514416313132, max

0.9797426844580166

In [54]:
#Hyperparameter Tuning of Logistic Regression
from sklearn.model_selection import GridSearchCV
log_reg = LogisticRegression(random_state=rs)
params = [    
    {'penalty' : ['l1'],
    'C' : [4.281332398719396],
    'solver' : ['liblinear'],
    'max_iter' : [1000]
   }
]

grid_search = GridSearchCV(estimator=log_reg,
                           param_grid=params,
                           cv=5, verbose=10, scoring = "f1")
grid_search.fit(processed_smote_train_data_trX, processed_smote_train_data_trY)
grid_search.best_score_

Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV 1/5; 1/1] START C=4.281332398719396, max_iter=1000, penalty=l1, solver=liblinear
[CV 1/5; 1/1] END C=4.281332398719396, max_iter=1000, penalty=l1, solver=liblinear;, score=0.980 total time=   1.3s
[CV 2/5; 1/1] START C=4.281332398719396, max_iter=1000, penalty=l1, solver=liblinear
[CV 2/5; 1/1] END C=4.281332398719396, max_iter=1000, penalty=l1, solver=liblinear;, score=0.981 total time=   0.8s
[CV 3/5; 1/1] START C=4.281332398719396, max_iter=1000, penalty=l1, solver=liblinear
[CV 3/5; 1/1] END C=4.281332398719396, max_iter=1000, penalty=l1, solver=liblinear;, score=0.980 total time=   0.8s
[CV 4/5; 1/1] START C=4.281332398719396, max_iter=1000, penalty=l1, solver=liblinear
[CV 4/5; 1/1] END C=4.281332398719396, max_iter=1000, penalty=l1, solver=liblinear;, score=0.982 total time=   1.1s
[CV 5/5; 1/1] START C=4.281332398719396, max_iter=1000, penalty=l1, solver=liblinear
[CV 5/5; 1/1] END C=4.281332398719396, max_iter=1000

0.9809223616367435

In [55]:
#Hyperparameter Tuning of Logistic Regression
from sklearn.model_selection import GridSearchCV
log_reg = LogisticRegression(random_state=rs)
params = [    
    {'penalty' : ['l1'],
    'C' : [0.012742749857031334],
    'solver' : ['liblinear'],
    'max_iter' : [1000]
   }
]

grid_search = GridSearchCV(estimator=log_reg,
                           param_grid=params,
                           cv=5, verbose=10, scoring = "f1")
grid_search.fit(processed_adasyn_train_data_trX, processed_adasyn_train_data_trY)
grid_search.best_score_

Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV 1/5; 1/1] START C=0.012742749857031334, max_iter=1000, penalty=l1, solver=liblinear
[CV 1/5; 1/1] END C=0.012742749857031334, max_iter=1000, penalty=l1, solver=liblinear;, score=0.969 total time=   0.5s
[CV 2/5; 1/1] START C=0.012742749857031334, max_iter=1000, penalty=l1, solver=liblinear
[CV 2/5; 1/1] END C=0.012742749857031334, max_iter=1000, penalty=l1, solver=liblinear;, score=0.970 total time=   0.5s
[CV 3/5; 1/1] START C=0.012742749857031334, max_iter=1000, penalty=l1, solver=liblinear
[CV 3/5; 1/1] END C=0.012742749857031334, max_iter=1000, penalty=l1, solver=liblinear;, score=0.970 total time=   0.5s
[CV 4/5; 1/1] START C=0.012742749857031334, max_iter=1000, penalty=l1, solver=liblinear
[CV 4/5; 1/1] END C=0.012742749857031334, max_iter=1000, penalty=l1, solver=liblinear;, score=0.970 total time=   0.6s
[CV 5/5; 1/1] START C=0.012742749857031334, max_iter=1000, penalty=l1, solver=liblinear
[CV 5/5; 1/1] END C=0.012

0.9278984247506564

In [57]:
from sklearn.decomposition import PCA
pca = PCA(n_components=35,random_state=rs)
pca.fit_transform(processed_adasyn_train_data_trX)
pca.explained_variance_ratio_.cumsum()

array([0.45617041, 0.61553262, 0.70532838, 0.773927  , 0.8194312 ,
       0.8565001 , 0.88107445, 0.90199282, 0.92086837, 0.93634785,
       0.94891693, 0.95998092, 0.9662788 , 0.97182296, 0.97665997,
       0.98117204, 0.98516076, 0.98841342, 0.99124403, 0.99387603,
       0.99536489, 0.99650467, 0.99755416, 0.99836565, 0.99895539,
       0.99937264, 0.99971784, 0.99980116, 0.99987277, 0.99992989,
       0.99996524, 0.99998084, 0.99999467, 0.99999801, 1.        ])